# LIBRARIES

In [29]:
import pandas as pd

import numpy as np

import os

from sklearn.linear_model import LinearRegression

# CONFIGURATION

In [30]:
sOutputSymbol = 'BTCUSD'
sModelType = 'MLP'

sFolderPath = 'Data/'+ sOutputSymbol +'//'+ sModelType + '//'
dfFullFactorialExperiments = pd.read_csv( sFolderPath + 'Full Factorial Design//Experiments.csv', index_col = 'Run ID')

In [31]:
iNumberOfIteration = 10
fDelta = 0.1

# STEEPEST DESCENT

In [32]:
dfFactors = dfFullFactorialExperiments.iloc[:, :-1]
dfResponse = dfFullFactorialExperiments['Response']
iMaxRunId = dfFullFactorialExperiments.index.max()

## Fit Linear Regression Model

In [33]:
oLinearRegression  = LinearRegression()
oLinearRegression.fit(dfFactors, dfResponse)

LinearRegression()

## Coefficient of Determination

In [34]:
oLinearRegression.score(dfFactors, dfResponse)

0.7299305810392182

## Equation

In [35]:
oLinearRegression.coef_

array([ 124.62818062, 2420.01582436])

In [36]:
oLinearRegression.intercept_

433838.2628621337

## Set New Experiments

In [37]:
aRatiosToDecrease = ((oLinearRegression.coef_)/(oLinearRegression.coef_[0]))*-1 #first factor is used as base factor
aRatiosToDecrease = aRatiosToDecrease[0] * fDelta

aCenterValues = np.array((dfFactors.max()+dfFactors.min())/2)
aIterations = range(0, iNumberOfIteration)
aNewDesign = aCenterValues + (np.array(np.add(aIterations, 1)).reshape(-1, 1) * aRatiosToDecrease * aCenterValues)
aNewDesign = aNewDesign.astype(np.int)

dfNewDesign =pd.DataFrame(
    data = aNewDesign,
    columns = dfFactors.columns,
    index = aIterations
)
ixExperimetnsLessThan10 = dfNewDesign[(dfNewDesign < 10).any(axis= 1)].index
dfNewDesign.loc[ixExperimetnsLessThan10]
dfNewDesign.drop(ixExperimetnsLessThan10, axis = 0 , inplace = True)

sDesignFolder = sFolderPath + '//Steepest Descent//'
os.makedirs(sDesignFolder, exist_ok = True)
dfNewDesign.to_csv(sDesignFolder +'Design.csv', index=True, index_label='Run ID')